# PE Validation Fall 20

In [25]:
# import necessary modules 
# please run this everytime you open it
import pandas as pd 
import numpy as np 
import matplotlib as plt 
import seaborn as sns
import re
import os

Task1:

In [2]:
# figure out a way to read in schema txt files (inside the schema folder)
# for example, I have a source file (such as Source_relevance.csv) and 
# I know the question_label, but I want to build a function to know 
# whether this question (Q1/Q2..) is multiple-choice or checklist
# build a function to accomplish this
# func(schemafile(...txt), input(..csv), question_label(Q1/Q2...)) -> checlist/multiple choice 

Task2: 

In [3]:
# not only do I want to know whether it's multiple-choice or checklist 
# I also want to know how many different answer choices are there
# build a function to accomplish this 
# func(schemafile(...txt), input(..csv), question_label(Q1/Q2...)) -> # of different answer choices

Task3:

In [4]:
# figure out how to uniquely identify each submission, 
# since there might be one contributor who made 
# several submissions. 
# can't use contributor_uuid. What to use? 

Task4: 

In [5]:
# one-hot-encode checklist questions to [0, 1]
# for example, Q4 in source relevance task is a checklist question
# if someone selects both A1 and A2, then it's [1, 1]
# return a dataframe that has each question_label but list out the answers in [] format. 

## Experimentation ground below: 

## Charlie

## Katherine

## Yewen

### Questions: 

In [6]:
# # read json lines file 
# data = pd.read_json('2020 data/2020-09-25_pe_webhooks.jsonl', lines=True)
# data.head()

In [7]:
# source.columns

In [8]:
# source[source["contributor_uuid"] == "00f548b7-6b63-4b47-828e-8e416b6ca0e2"][["contributor_uuid", "created", "finish_time", "elapsed_seconds", "question_label", "answer_label", "answer_uuid"]]

What's going on here? 

What can we do about multi-choice? <br> 
What other questions we are expected to address? <br> 

# K's algorithm 

Now only try working with multiple-choice nominal questions <br>
Example, Q2 in Source Relevance Specialist task

In [9]:
# only select rows that correspond to question 2 
# ques_two_b = source[source["question_label"] == "T1.Q2"]

In [10]:
# ques_two_b.columns

In [11]:
# # experiment to find unique identifier 
# ex = source[source["contributor_uuid"] == "00f548b7-6b63-4b47-828e-8e416b6ca0e2"]
# ex = ex.loc[:, ["article_filename", "quiz_task_uuid", "contributor_uuid", "tua_uuid", "question_label", "answer_label", "submitted_tua_uuid", "answer_uuid"]]
# ex

**tua_uuid can be used as a unique identifier!!!**

**can't use article_filename, since the same article can have multiple questions!!!**

**the same article can have different tasks (quiz_task_uuid) and different contributors.**

In [12]:
# # let's see article filename
# ques_two_b["article_filename"].unique()

In [13]:
# # right now I will use different articles same question as an example
# # might be wrong
# # select tua_uuid as different observers 
# pre = ques_two_b.loc[:, ["quiz_task_uuid", "contributor_uuid", "question_label", "answer_label"]]
# pre.head()

Construct the **reliability data matrix**

In [14]:
# # convert the df to pivot_table 
# pv = pd.pivot_table(data=pre, index="contributor_uuid", columns="quiz_task_uuid", values="answer_label", aggfunc=list)
# pv

Geez, a lot of missing values! 

**construct a coincidence matrix**

## Triager to uAlpha

In [92]:
# create mapping for topic_name
dic = {"Quoted Sources":1,
           "Arguments":2,
           "Assertions":3, 
           "Needs Fact-Check":4, 
           "Language":5, 
           "Reasoning":6, 
           "Probability":7, 
           "Evidence":8}

    
# mapping topic_name to integers 
def mapping(df, dic):
    # df - dataframe to be mapped 
    # dic - dictionary of mapping 
    df["topic_name"] = df["topic_name"].map(dic)
    return df 

# add adjusted start_pos and adjusted end_pos for df 
def adjust(df):
    # df - a dataframe with ["article_number", "article_text_length", "start_pos", "end_pos"] cols
    article_number = df["article_number"]
    article_text_length = df["article_text_length"]
    start_pos = df["start_pos"]
    end_pos = df["end_pos"]
    # the cumulative sum of article_text_length
    cumulative = []
    start_pos_adjusted = []
    end_pos_adjusted = []
    pre_total = 0
    total = article_text_length[0]
    cur_num = article_number[0]
    for i in np.arange(len(article_number)):
        if article_number[i] == cur_num:
            total = total + 0
            cumulative.append(total)
        else:
            pre_total = total
            cur_num = article_number[i]
            total = total + article_text_length[i]
            cumulative.append(total)
        start_pos_adjusted.append(start_pos[i] + pre_total)
        end_pos_adjusted.append(end_pos[i] + pre_total)
    df["cumulative"] = cumulative
    df["start_pos_adjusted"] = start_pos_adjusted
    df["end_pos_adjusted"] = end_pos_adjusted
    return df 


# filter df and select only certain cols as input
def slice_input(df):
    return df.loc[:, ["contributor_uuid", "topic_name", "start_pos"
                                    , "end_pos"
                                    , "article_text_length"
                                    , "article_number"]]


# filter df and select only certain cols as output
def slice_output(df):
    return df.loc[:, ["index", "contributor_uuid", 
                               "topic_name", 
                               "blank", 
                               "start_pos_adjusted", 
                               "end_pos_adjusted", 
                               "cumulative"]]
    

# read in a directory that contains Triager csvs 
# split each Triager csv to 4 other csvs based on topic_name (argument, reasoning, etc)
# and write it as csvs in the output directory
def triager_split(input_directory, output_directory):
    # input_directory - e.g. "Triager data"
    # output_directory - e.g. "Triager output"
    for file in os.listdir(input_directory):
        triager_split_help(input_directory, file, output_directory)
    print("Triager Tranformation Done! Ready to be imported to uAlpha!")
    
# take in a Triager csv and split it to 4 different csvs 
# based on topic_name 
def triager_split_help(input_directory, fileName, output_directory):
    # input_directory - e.g. "Triager data"
    # fileName - e.g. "Covid_Form1.0.adjudicated-2020-10-04T2314-Tags.csv"
    # output_directory - e.g. "Triager output"
    name = os.path.join(input_directory, fileName)
    df = pd.read_csv(name)
    # different topic_names 
    topic_names = df["topic_name"].unique()
    for topic in topic_names: 
        # select all rows for that specific topic name 
        df_topic = df.loc[df["topic_name"] == topic]
        # re-index such that its index starts from 0 again
        df_topic.reset_index(drop=True, inplace=True)
        # select only these columns 
        filtered = slice_input(df_topic)
        # slice contributor_uuid with the first 6 chars 
        filtered["contributor_uuid"] = filtered["contributor_uuid"].str[:6]
        # map topic_name to integers
        mapping(filtered, dic)
        # add one column of u + str(row)
        index = 'u' + pd.Series(filtered.index).astype(str)
        filtered.insert(loc=0, column="index", value=index)
        # add a blank column 
        filtered.insert(loc=3, column="blank", value="")
        # add modified columns
        filtered = adjust(filtered)
        # filter and select only certain columns 
        output = slice_output(filtered)
        # the batch_name, e.g. "Covid"
        batch_name = re.split(r'_', fileName)[0]
        name = '{0}-Triager-{1}.csv'.format(batch_name, topic)
        path = os.path.join(output_directory, name)
        # write to csvs
        output.to_csv(path)
    return None


In [93]:
triager_split("Triager data", "Triager output")

Triager Tranformation Done! Ready to be imported to uAlpha!
